In [561]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [562]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

#  **Understand the data**

In [563]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [564]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [565]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [566]:
train_data.isnull()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,False,False,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
886,False,False,False,False,False,False,False,False,False,False,True,False
887,False,False,False,False,False,False,False,False,False,False,False,False
888,False,False,False,False,False,True,False,False,False,False,True,False
889,False,False,False,False,False,False,False,False,False,False,False,False


# **Pre-processing**

In [567]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [568]:
#dropping null values of embarked ,  Only 2 instances of this in training and 0 in test
train_data.dropna(subset=['Embarked'],inplace = True)

In [569]:
# check dropping is ok
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [570]:
# impute nulls for age records
median_age= pd.concat([test_data,train_data]).Age.median()
train_data.Age.fillna(median_age,inplace=True)
test_data.Age.fillna(median_age,inplace=True)

In [571]:
# recheck 
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [572]:
# filling Fare in test data
test_data.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [573]:
test_data.Fare.describe()

count    417.000000
mean      35.627188
std       55.907576
min        0.000000
25%        7.895800
50%       14.454200
75%       31.500000
max      512.329200
Name: Fare, dtype: float64

In [574]:
median=train_data.Fare.median()
test_data.Fare.fillna(median,inplace=True)

In [575]:
# recheck 
test_data.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

In [576]:
train_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,28.0,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [577]:
train_data.Cabin.describe()

count     202
unique    146
top        G6
freq        4
Name: Cabin, dtype: object

# **Adding features **

In [578]:
# null values of Cabin get 0 while others gets the number of cabins they exist in ex  passengerid:28  exissts in 3 cabins
train_data['cabin_multiple'] = train_data.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))
test_data['cabin_multiple'] = test_data.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))

In [579]:
# store the initials of Cabin record indicating cabin type while nulls stored as n
train_data['cabin_adv'] = train_data.Cabin.apply(lambda x: str(x)[0])
test_data['cabin_adv'] = test_data.Cabin.apply(lambda x: str(x)[0])

In [580]:
# classifyng names based on titles
train_data['title'] = train_data.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())
test_data['title'] = test_data.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())

In [581]:
train_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,cabin_multiple,cabin_adv,title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,n,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,n,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,C,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,n,Mr
5,6,0,3,"Moran, Mr. James",male,28.0,0,0,330877,8.4583,NaN,Q,0,n,Mr
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,1,E,Mr
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,0,n,Master
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,0,n,Mrs
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,0,n,Mrs


In [582]:
train_data['train']=1  # added in order to split later train from test
test_data['train']=0
data=pd.concat([train_data,test_data])  #  concatenated to unify the number of features 
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked','cabin_multiple', 'cabin_adv','title','train']


In [583]:
#creating dummy variables from categories 

x=pd.get_dummies(data[features])
x=x[x.train==1].drop(['train'],axis=1)
x.head()

,Pclass,Age,SibSp,Parch,Fare,cabin_multiple,Sex_female,Sex_male,Embarked_C,Embarked_Q,...,title_Master,title_Miss,title_Mlle,title_Mme,title_Mr,title_Mrs,title_Ms,title_Rev,title_Sir,title_the Countess
0,3,22.0,1,0,7.2500,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,38.0,1,0,71.2833,1,1,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,3,26.0,0,0,7.9250,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,35.0,1,0,53.1000,1,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,3,35.0,0,0,8.0500,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


In [584]:
# target object
y =train_data['Survived']
# Split into validation and training data
train_X, test_X, train_y, test_y = train_test_split(x,y, random_state = 1)

# **Model Building**

In [585]:
#    RandomForestClassifier 

In [586]:

rf = RandomForestClassifier(random_state = 1)
rf.fit(train_X, train_y)
rf_prediction = rf.predict(test_X)
accuracy = accuracy_score(test_y,rf_prediction)
print(accuracy)


0.8295964125560538


In [587]:
cv = cross_val_score(rf,train_X,train_y,cv=5)
print(cv)
print(cv.mean())

[0.82835821 0.78195489 0.80451128 0.80451128 0.80451128]
0.8047693861519469


In [588]:
# DecisionTreeClassifie

In [589]:
dt = DecisionTreeClassifier(random_state=0)
dt.fit(train_X, train_y)
scores = cross_val_score(dt, x, y, cv=5)
print(scores)
accuracy = scores.mean()
print("\n accuracy: ",accuracy)

[0.75280899 0.78089888 0.83146067 0.76404494 0.81355932]

 accuracy:  0.7885545610359931


In [590]:
# Soft Voting Classifier

In [591]:
svc = SVC()
svc.fit(train_X, train_y)
svc_prediction = svc.predict(test_X)
accuracy = accuracy_score(test_y,svc_prediction)
print(accuracy)

0.6457399103139013


In [592]:
# LogisticRegression

In [593]:
lr = LogisticRegression(max_iter=1000)
lr.fit(train_X, train_y)
lr_prediction = lr.predict(test_X)
accuracy = accuracy_score(test_y,lr_prediction)
print(accuracy)

0.8565022421524664


In [594]:

scores = cross_val_score(lr, x, y, cv=10)
print(scores)
accuracy = scores.mean()
print("\n accuracy: ",accuracy)

[0.79775281 0.84269663 0.7752809  0.87640449 0.83146067 0.7752809
 0.80898876 0.82022472 0.86516854 0.85227273]

 accuracy:  0.824553115423902


#  ** LogisticRegression  gives the highest score**

In [595]:
test = pd.get_dummies(data[features])
test=test[test.train==0].drop(['train'],axis=1)
test.head()
prediction = lr.predict(test)

In [596]:
submission = pd.DataFrame({"PassengerId":test_data.PassengerId, 'Survived':prediction.astype(int).ravel()})
submission.to_csv("final_submission.csv",index = False)
